In [42]:
import pandas as pd
import numpy as np
import re

df = pd.read_csv('pbp_raw/0022200002_pbp.csv',index_col=0)

def str_to_tuple(s):
    return tuple(s.strip('() ').replace(' ', '').split(','))

df['h_lineup'] = df.h_lineup.apply(lambda x: str_to_tuple(x))
df['a_lineup'] = df.a_lineup.apply(lambda x: str_to_tuple(x))

eventdict = {1:'FGM',
             2:'FGA',
             3:'FTA',
             4:'TRB',
             5:'TURNOVER',
             6:'FOUL',
             8:'SUB',
             9:'TIMEOUT',
             10:'JUMP_BALL',
             12:'Q_START',
             13:'Q_END',
             18:'REPLAY'             
            }
cols = ['game_id', 'lineup','player_id','PTS','FGM', 'FGA', '3PM', '3PA', 'FTM', 'FTA', 'REB', 'OREB', 'DREB', 'AST', 'STL', 'BLK', 'TO', 'PF', 'MIN', 'POSS']


In [43]:
#allll this ugly stuff just to get a column called 'rebound_type'
mask = (df.eventmsgtype == 4) & (~df.homedescription.isnull() | ~df.visitordescription.isnull())
pattern = r'Off:(\d+) Def:(\d+)'
#getting rebound counts from play description and filling nulls, and casting to int
df[['offensive_rebounds', 'defensive_rebounds']] = df['homedescription'].str.extract(pattern)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df['visitordescription'].str.extract(pattern)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].fillna(0)
df[['offensive_rebounds', 'defensive_rebounds']] = df[['offensive_rebounds', 'defensive_rebounds']].astype(int)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].fillna(0)
df[['offensive_rebounds_v', 'defensive_rebounds_v']] = df[['offensive_rebounds_v', 'defensive_rebounds_v']].astype(int)

df.sort_values(by=['player1_id','period','total_elapsed_time','eventnum'], inplace=True)

#comparing reb total to previous to check what type of rebound occurred
df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df['rebound_type'] = ''
df.loc[(df.defensive_rebounds > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.loc[(df.defensive_rebounds_v > df.def_reb_previous), 'rebound_type'] = 'DRB'
df.loc[(df.offensive_rebounds_v > df.off_reb_previous), 'rebound_type'] = 'ORB'
df.drop([ 'offensive_rebounds', 'defensive_rebounds',
       'offensive_rebounds_v', 'defensive_rebounds_v', 'def_reb_previous',
       'off_reb_previous'],inplace=True,axis=1)
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_14284\3264647376.py:15: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['def_reb_previous'] = df[mask].groupby('player1_id')['defensive_rebounds'].shift(fill_value=0)
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_14284\3264647376.py:16: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df['off_reb_previous'] = df[mask].groupby('player1_id')['offensive_rebounds'].shift(fill_value=0)


In [44]:
fga_filter = (df.eventmsgtype==1) | (df.eventmsgtype==2)
two_pt_filter = (df.eventmsgtype==1) & ~((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_pt_filter = (df.eventmsgtype==1) & ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
three_a_filter = ((df.homedescription.str.contains('3PT',na=False)) | (df.visitordescription.str.contains('3PT',na=False)))
ft_filter = (df.eventmsgtype==3) & ((df.homedescription.str.contains('PTS',na=False)) | (df.visitordescription.str.contains('PTS',na=False)))
fta_filter = df.eventmsgtype==3 
assist_filter = (df.eventmsgtype==1) & ~df.player2_name.isnull()

orb_filter = df.rebound_type == 'ORB'
drb_filter = df.rebound_type == 'DRB'
trb_filter = (df.rebound_type == 'DRB') | (df.rebound_type == 'ORB')
stl_filter = df.homedescription.str.contains('STEAL',na=False) | df.visitordescription.str.contains('STEAL',na=False)
blk_filter = df.homedescription.str.contains('BLOCK',na=False) | df.visitordescription.str.contains('BLOCK',na=False)
to_filter = df.eventmsgtype==5
foul_filter = df.eventmsgtype==6
time_filter = (df.play_elapsed_time != 0)

In [45]:
made_last_ft_pattern = r'(1 of 1|2 of 2|3 of 3).*PTS'
visitor_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.homedescription.isna())) | # home team drb means visitor poss ends
(df.homedescription.str.contains('STEAL',na=False)) |# home team steal means visitor poss ends
((df.eventmsgtype==1) & ~(df.visitordescription.isna())) |# visitor team makes shot means visitor poss ends
((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) |#visitor team MAKES last ft means poss ends
((df.eventmsgtype == 5) & (df.homedescription.isna())) #unforced visitor turnover
) 

home_poss_filter = (
((df.rebound_type == 'DRB') & ~(df.visitordescription.isna())) | # home team drb means visitor poss ends
(df.visitordescription.str.contains('STEAL',na=False)) |
((df.eventmsgtype==1) & ~(df.homedescription.isna())) |
((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |
((df.eventmsgtype == 5) & (df.visitordescription.isna()))
) 
df['home_poss'],df['away_poss'] = np.nan,np.nan
df.loc[visitor_poss_filter,'away_poss'] = 1
df.loc[home_poss_filter,'home_poss'] = 1

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_14284\3561570407.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.visitordescription.str.contains(made_last_ft_pattern)) |#visitor team MAKES last ft means poss ends
C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_14284\3561570407.py:14: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  ((df.eventmsgtype==3) & df.homedescription.str.contains(made_last_ft_pattern)) |


In [46]:
# Determine the most recent possession
def determine_possession(row):
    if row['home_poss'] == 1:
        return 'home'
    elif row['away_poss'] == 1:
        return 'away'

# Apply the function to each row to create a new 'new_col' column
df['has_ball'] = df.apply(determine_possession, axis=1)

# Fill NaN values in the 'new_col' column based on the previous row
df['has_ball'] = df['has_ball'].fillna(method='bfill')

In [47]:
def apply_stat_to_lineup(pbp_df,filter,stat_type,stat_value,a_or_v):
    game_id = df.game_id[0]
    box_score = pd.read_csv(f'box_scores/box_00{game_id}.csv',index_col=0)
    box = box_score[['player_id','player_name']].values.tolist()
    id_to_name = {id:name for id,name in box}

    filtered_df = df.loc[filter].copy()
    filtered_df[f'{a_or_v}_lineup_copy'] = filtered_df[f'{a_or_v}_lineup']
    exploded_df = filtered_df.explode(f'{a_or_v}_lineup_copy', ignore_index=True)
    exploded_df.rename(columns={f'{a_or_v}_lineup_copy': 'poss_player_id'}, inplace=True)
    exploded_df['poss_player_name'] = exploded_df.apply(lambda x: id_to_name[int(x.poss_player_id)],axis=1)
    exploded_df_small = exploded_df[['game_id','eventnum','poss_player_id','poss_player_name','h_lineup']].copy()
    exploded_df_small['poss_player_id'] = exploded_df_small['poss_player_id'].astype(int)
    exploded_df_small['stat'] = stat_type
    exploded_df_small['value'] = stat_value
    return exploded_df_small.values.tolist()

poss_list = apply_stat_to_lineup(df,home_poss_filter,'poss',1,a_or_v='h')#h or a
poss_list += apply_stat_to_lineup(df,visitor_poss_filter,'poss',1,a_or_v='a')#h or a
poss_list

[[22200002,
  11,
  201939,
  'Stephen Curry',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  11,
  202691,
  'Klay Thompson',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  11,
  203110,
  'Draymond Green',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  11,
  203952,
  'Andrew Wiggins',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  11,
  1626172,
  'Kevon Looney',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  21,
  201939,
  'Stephen Curry',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  21,
  202691,
  'Klay Thompson',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  21,
  203110,
  'Draymond Green',
  ('201939', '202691', '203110', '203952', '1626172'),
  'poss',
  1],
 [22200002,
  21,
  203952,
  'Andrew Wiggins',
  ('2

In [48]:
def get_time_credit_list(df):
    game_id = df.game_id[0]
    box_score = pd.read_csv(f'box_scores/box_00{game_id}.csv',index_col=0)
    box = box_score[['player_id','player_name']].values.tolist()
    id_to_name = {id:name for id,name in box}

    temp = df.groupby(['h_lineup'])['play_elapsed_time'].sum()
    lineup_times = temp.loc[temp != 0].sort_values(ascending=False)
    lineup_times_list = [(index, value) for index, value in lineup_times.items()]

    temp = df.groupby(['a_lineup'])['play_elapsed_time'].sum()
    lineup_times = temp.loc[temp != 0].sort_values(ascending=False)
    lineup_times_list += [(index, value) for index, value in lineup_times.items()]

    time_credit = []
    for lineup,time in lineup_times_list:
        for player in lineup:
            time_credit.append([game_id,np.nan,int(player),id_to_name[int(player)],lineup,'sec',time])
    return time_credit

In [49]:
df.sort_values(by=['period','total_elapsed_time','eventnum'],inplace=True)
df.head(10)

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,pctimestring,homedescription,neutraldescription,visitordescription,score,...,play_elapsed_time,total_elapsed_time,max_time,time_remaining,h_lineup,a_lineup,rebound_type,home_poss,away_poss,has_ball
0,22200002,2,12,0,1,12:00,NaN,Start of 1st Period (10:41 PM EST),NaN,NaN,...,0.0,0.0,2880.0,2880.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,away
1,22200002,4,10,0,1,12:00,Jump Ball Looney vs. Davis: Tip to James,NaN,NaN,NaN,...,0.0,0.0,2880.0,2880.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,away
2,22200002,7,6,1,1,11:43,Looney P.FOUL (P1.T1) (T.Brothers),NaN,NaN,NaN,...,17.0,17.0,2880.0,2863.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,away
3,22200002,9,1,6,1,11:37,NaN,NaN,Davis 3' Driving Layup (2 PTS),2 - 0,...,6.0,23.0,2880.0,2857.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,1.0,away
4,22200002,10,2,78,1,11:21,MISS Wiggins 9' Floating Jump Shot,NaN,NaN,NaN,...,16.0,39.0,2880.0,2841.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,home
5,22200002,11,4,0,1,11:17,NaN,NaN,Westbrook REBOUND (Off:0 Def:1),NaN,...,4.0,43.0,2880.0,2837.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",DRB,1.0,NaN,home
6,22200002,12,2,1,1,11:13,NaN,NaN,MISS Beverley 3PT Jump Shot,NaN,...,4.0,47.0,2880.0,2833.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,away
7,22200002,13,4,0,1,11:09,D. Green REBOUND (Off:0 Def:1),NaN,NaN,NaN,...,4.0,51.0,2880.0,2829.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",DRB,NaN,1.0,away
8,22200002,16,6,2,1,10:59,NaN,NaN,Beverley S.FOUL (P1.T1) (S.Twardoski),NaN,...,10.0,61.0,2880.0,2819.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,home
9,22200002,18,3,13,1,10:59,Thompson Free Throw 1 of 3 (1 PTS),NaN,NaN,2 - 1,...,0.0,61.0,2880.0,2819.0,"(201939, 202691, 203110, 203952, 1626172)","(2544, 201566, 201976, 203076, 1629022)",,NaN,NaN,home


In [50]:
df.columns

Index(['game_id', 'eventnum', 'eventmsgtype', 'eventmsgactiontype', 'period',
       'pctimestring', 'homedescription', 'neutraldescription',
       'visitordescription', 'score', 'scoremargin', 'player1_id',
       'player1_name', 'player1_team_abbreviation', 'player2_id',
       'player2_name', 'player2_team_abbreviation', 'player3_id',
       'player3_name', 'player3_team_abbreviation', 'timeinseconds',
       'play_elapsed_time', 'total_elapsed_time', 'max_time', 'time_remaining',
       'h_lineup', 'a_lineup', 'rebound_type', 'home_poss', 'away_poss',
       'has_ball'],
      dtype='object')

In [51]:
def get_stat_df(pbp_df,filter,stat_type,stat_value,player_num='1'):
    filtered_df = pbp_df.loc[filter][['game_id','eventnum',f'player{player_num}_id',f'player{player_num}_name','h_lineup']]
    filtered_df['stat'] = stat_type
    filtered_df['value'] = stat_value
    return filtered_df.values.tolist()

#results is a list of every stat credit for every play
results = get_stat_df(df,two_pt_filter,'PTS',2)
results += get_stat_df(df,three_pt_filter,'PTS',3)
results += get_stat_df(df,ft_filter,'PTS',1)

results = get_stat_df(df,two_pt_filter,'FGM',1)
results += get_stat_df(df,three_pt_filter,'3PM',1)
results += get_stat_df(df,ft_filter,'FTM',1)

results += get_stat_df(df,fga_filter,'FGA',1)
results += get_stat_df(df,fta_filter,'FTA',1)
results += get_stat_df(df,three_a_filter,'3PA',1)

results += get_stat_df(df,trb_filter,'TRB',1)
results += get_stat_df(df,drb_filter,'DRB',1)
results += get_stat_df(df,orb_filter,'ORB',1)

results += get_stat_df(df,stl_filter,'STL',1,player_num=2)
results += get_stat_df(df,blk_filter,'BLK',1,player_num=3)
results += get_stat_df(df,to_filter,'TOV',1,player_num=1)
results += get_stat_df(df,foul_filter,'PF',1)

results += get_stat_df(df,assist_filter,'AST',1,player_num='2')

results += apply_stat_to_lineup(df,home_poss_filter,'poss',1,a_or_v='h')#h or a
results += apply_stat_to_lineup(df,visitor_poss_filter,'poss',1,a_or_v='a')#h or a
results += get_time_credit_list(df)
results[-1]

[22200002,
 nan,
 1629134,
 'Kendrick Nunn',
 ('2544', '201566', '201976', '203076', '1629022', '1629134'),
 'sec',
 12.0]

In [52]:
df.loc[df.h_lineup == ('201939', '202691', '203110', '1626172', '1629673')]

,game_id,eventnum,eventmsgtype,eventmsgactiontype,period,pctimestring,homedescription,neutraldescription,visitordescription,score,...,play_elapsed_time,total_elapsed_time,max_time,time_remaining,h_lineup,a_lineup,rebound_type,home_poss,away_poss,has_ball
284,22200002,416,8,0,2,0:01,SUB: Looney FOR Wiggins,NaN,NaN,NaN,...,0.0,1439.0,2880.0,1441.0,"(201939, 202691, 203110, 1626172, 1629673)","(2544, 201566, 203076, 1629022, 1630346)",,NaN,NaN,away
285,22200002,418,3,12,2,0:01,NaN,NaN,James Free Throw 2 of 2 (14 PTS),52 - 59,...,0.0,1439.0,2880.0,1441.0,"(201939, 202691, 203110, 1626172, 1629673)","(2544, 201566, 203076, 1629022, 1630346)",,NaN,1.0,away
286,22200002,419,13,0,2,0:00,NaN,End of 2nd Period (11:52 PM EST),NaN,52 - 59,...,1.0,1440.0,2880.0,1440.0,"(201939, 202691, 203110, 1626172, 1629673)","(2544, 201566, 203076, 1629022, 1630346)",,NaN,NaN,home


In [53]:
play_credit_df = pd.DataFrame(results,columns=['game_id','eventnum','player_id','player_name','lineup','stat_type','stat_value'])
play_credit_df.groupby(by=['player_id', 'stat_type','player_name'])['stat_value'].sum()

player_id  stat_type  player_name 
2544       3PA        LeBron James     10.0
           3PM        LeBron James      3.0
           AST        LeBron James      8.0
           DRB        LeBron James      2.0
           FGA        LeBron James     26.0
                                      ...  
1631306    DRB        Cole Swider       1.0
           FGA        Cole Swider       1.0
           TRB        Cole Swider       1.0
           poss       Cole Swider       4.0
           sec        Cole Swider     112.0
Name: stat_value, Length: 273, dtype: float64

In [54]:
game_play_agg = play_credit_df.groupby(by=['lineup','player_id', 'stat_type','player_name'],as_index=False)['stat_value'].sum()
game_play_agg

,lineup,player_id,stat_type,player_name,stat_value
0,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,3PA,LeBron James,2.0
1,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,FGA,LeBron James,3.0
2,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,FGM,LeBron James,1.0
3,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,ORB,LeBron James,1.0
4,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,TOV,LeBron James,1.0
...,...,...,...,...,...
920,"(2544, 203076, 1629022, 1629134, 1630559)",2544,sec,LeBron James,160.0
921,"(2544, 203076, 1629022, 1629134, 1630559)",203076,sec,Anthony Davis,160.0
922,"(2544, 203076, 1629022, 1629134, 1630559)",1629022,sec,Lonnie Walker IV,160.0
923,"(2544, 203076, 1629022, 1629134, 1630559)",1629134,sec,Kendrick Nunn,160.0


In [55]:
pivoted_game = game_play_agg.pivot(index=['lineup', 'player_id', 'player_name'], columns='stat_type', values='stat_value').fillna(0).reset_index()
pivoted_game

stat_type,lineup,player_id,player_name,3PA,3PM,AST,BLK,DRB,FGA,FGM,FTA,FTM,ORB,PF,STL,TOV,TRB,poss,sec
0,"(1628978, 1629673, 1630164, 1630228, 1630541)",2544,LeBron James,2.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0
1,"(1628978, 1629673, 1630164, 1630228, 1630541)",201976,Patrick Beverley,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
2,"(1628978, 1629673, 1630164, 1630228, 1630541)",1628978,Donte DiVincenzo,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,4.0,124.0
3,"(1628978, 1629673, 1630164, 1630228, 1630541)",1629022,Lonnie Walker IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
4,"(1628978, 1629673, 1630164, 1630228, 1630541)",1629117,Wenyen Gabriel,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,"(2544, 203076, 1629022, 1629134, 1630559)",2544,LeBron James,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0
322,"(2544, 203076, 1629022, 1629134, 1630559)",203076,Anthony Davis,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0
323,"(2544, 203076, 1629022, 1629134, 1630559)",1629022,Lonnie Walker IV,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0
324,"(2544, 203076, 1629022, 1629134, 1630559)",1629134,Kendrick Nunn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,160.0


In [56]:
temp = pivoted_game.groupby(['player_id','player_name'],as_index=False).sum()
temp

C:\Users\Shared.Westover\AppData\Local\Temp\ipykernel_14284\365331223.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  temp = pivoted_game.groupby(['player_id','player_name'],as_index=False).sum()


stat_type,player_id,player_name,3PA,3PM,AST,BLK,DRB,FGA,FGM,FTA,FTM,ORB,PF,STL,TOV,TRB,poss,sec
0,2544,LeBron James,10.0,3.0,8.0,0.0,2.0,26.0,9.0,4.0,4.0,13.0,2.0,0.0,5.0,15.0,83.0,2112.0
1,201566,Russell Westbrook,3.0,1.0,3.0,0.0,6.0,12.0,6.0,5.0,4.0,5.0,1.0,1.0,4.0,11.0,73.0,1841.0
2,201939,Stephen Curry,13.0,4.0,7.0,1.0,5.0,22.0,6.0,9.0,9.0,1.0,1.0,4.0,4.0,6.0,74.0,1968.0
3,201976,Patrick Beverley,5.0,1.0,2.0,2.0,1.0,5.0,0.0,0.0,0.0,1.0,5.0,4.0,1.0,2.0,51.0,1480.0
4,202691,Klay Thompson,6.0,2.0,3.0,0.0,2.0,13.0,4.0,5.0,4.0,0.0,3.0,1.0,2.0,2.0,42.0,1179.0
5,203076,Anthony Davis,3.0,0.0,0.0,1.0,6.0,22.0,10.0,9.0,7.0,0.0,2.0,4.0,3.0,6.0,88.0,2136.0
6,203110,Draymond Green,0.0,0.0,5.0,0.0,5.0,5.0,2.0,0.0,0.0,0.0,1.0,0.0,1.0,5.0,54.0,1481.0
7,203210,JaMychal Green,3.0,2.0,0.0,0.0,2.0,6.0,1.0,0.0,0.0,4.0,1.0,1.0,0.0,6.0,46.0,1406.0
8,203952,Andrew Wiggins,7.0,4.0,4.0,1.0,5.0,14.0,4.0,0.0,0.0,1.0,5.0,1.0,1.0,6.0,70.0,1768.0
9,1626172,Kevon Looney,0.0,0.0,2.0,0.0,4.0,4.0,3.0,1.0,1.0,2.0,2.0,1.0,0.0,6.0,53.0,1268.0


In [57]:
pivoted_game.loc[pivoted_game.player_id == '201939']

stat_type,lineup,player_id,player_name,3PA,3PM,AST,BLK,DRB,FGA,FGM,FTA,FTM,ORB,PF,STL,TOV,TRB,poss,sec
